# LAB 5

* Make a rag or bq database and embed the documents
* Make api tool calling for weather and alaska city location
---
* Test and verify you can send a request and get a weather report or faq on the alaskan department of snow.
* Evaluate groundedness?
---
* Prompt filtering and response validation implemented into the solution (Keep to security guidlines) Kinda tied in to above testing
---
* Log conversations... Somewhere...
---
* Deploy onto web app
(utalize ai and quick assist to help with front end)


## SET UP
---

In [2]:
! pip install --upgrade --quiet google-cloud-aiplatform google-cloud-aiplatform[evaluation]
! pip install --quiet ipytest
! pip install --upgrade google-genai
! pip install google-cloud-modelarmor
! pip install --upgrad google-cloud-bigquery
!pip install requests==2.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 111.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 121.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 53.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colabsqlviz 0.2.7 requires protobuf<7.0.0,>=6.31.1, but you have protobuf 5.29.5 which is incompatible

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.2/134.2 kB 4.4 MB/s eta 0:00:00



Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

ambiguous option: --upgrad (--upgrade, --upgrade-strategy?)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.31.0 which is incompatible.
google-adk 1.17.0 requires requests<3.0.0,>=2.32.4, but you have requests 2.31.0 which is incompatible.
da


Copy over data
```
gsutil cp -r gs://labs.roitraining.com/alaska-dept-of-snow ~/
```

Create bq data files
```
export mydataset="alaska_dept_faq_db"
export mytable="faqs_table"
```
```
bq mk $mydataset
bq load \
  --source_format=CSV \
  --skip_leading_rows=0 \
  $mydataset.$mytable \
  ~/alaska-dept-of-snow/alaska-dept-of-snow-faqs.csv \
  question:STRING,answer:STRING

bq ls $mydataset

```

## Restart Session

You have set up env and datasets, you may need to restart before procededing with the rest of the code

In [20]:
import os
import pandas as pd
from IPython.display import display, Markdown, HTML

import requests

import pytest
import ipytest
ipytest.autoconfig()

from vertexai.evaluation import (
    MetricPromptTemplateExamples,
    EvalTask,
)

from google import genai
from google.genai.types import Tool, ToolCodeExecution, GenerateContentConfig, SafetySetting, Part, FunctionDeclaration
from google.cloud import modelarmor_v1
from google.api_core.client_options import ClientOptions

import datetime
from google.cloud import bigquery


import google.cloud.logging
import logging
import time


In [ ]:
project_id = !gcloud config get project
project_id = project_id[0]
location = "us-central1"
MY_API_KEY=""

model = "gemini-2.5-flash"


mydataset = "alaska_dept_faq_db"#!echo $mydataset
mytable = f"""{project_id}.{mydataset}.faqs_table""" #!echo $project_id.$mydataset.$mytable

mymodel = f"""{project_id}.{mydataset}.sql_connected_model"""
embededTable = f"""{project_id}.{mydataset}.withEmbedding"""

print(mymodel)
print(mytable)
print(embededTable)

qwiklabs-gcp-02-47840be12dd1.alaska_dept_faq_db.sql_connected_model
qwiklabs-gcp-02-47840be12dd1.alaska_dept_faq_db.faqs_table
qwiklabs-gcp-02-47840be12dd1.alaska_dept_faq_db.withEmbedding


In [26]:
# init genai sdk client
genai_client = genai.Client(
    vertexai=True,
    project=project_id,
    location=location
)

# init big query client
bq_client = bigquery.Client(project=project_id)
datasets = list(bq_client.list_datasets())


# init the Cloud Logging client
from google.cloud import logging as cloud_logging
log_client = google.cloud.logging.Client()
log_client.setup_logging()

log_handler = log_client.get_default_handler()

cloud_logger = logging.getLogger("cloudLogger")
cloud_logger.setLevel(logging.INFO)
cloud_logger.addHandler(log_handler)


## MODEL ARMOR
in the navigation menu
Security > ModelArmor
create a template and put the template path in the id below.

name == ADS_model_armor

In [4]:
armor_id = f"projects/{project_id}/locations/{location}/templates/ADS_model_armor"


armoredClient = modelarmor_v1.ModelArmorClient(
    transport="rest",
    client_options=ClientOptions(
        api_endpoint=f"modelarmor.{location}.rep.googleapis.com"
    ),
)

## Making Tables

In [6]:
## Makes a model that is connected to BigQuerry...
makeModel = f"""CREATE OR REPLACE MODEL `{mymodel}`
REMOTE WITH CONNECTION DEFAULT
OPTIONS (ENDPOINT = 'text-embedding-005');"""
print(makeModel)
bq_client.query(makeModel)

CREATE OR REPLACE MODEL `qwiklabs-gcp-02-47840be12dd1.alaska_dept_faq_db.sql_connected_model`
REMOTE WITH CONNECTION DEFAULT
OPTIONS (ENDPOINT = 'text-embedding-005');


QueryJob<project=qwiklabs-gcp-02-47840be12dd1, location=US, id=f3004b7f-ecbd-49d1-95d9-7456768f8707>

In [17]:
## Makes a Table that has been embeded
embedding = f"""
CREATE OR REPLACE TABLE `{embededTable}` AS
  SELECT *
  FROM ML.GENERATE_EMBEDDING(
  MODEL `{mymodel}`,
  (
    SELECT question, answer,	CONCAT(question, answer) AS content
    FROM `{mytable}`)
  );
"""
query_job = bq_client.query(embedding)

results = query_job.result()
print(results)
print(query_job)
if results == None:
  print("Big Query Client isn't working")

QueryJob<project=qwiklabs-gcp-02-47840be12dd1, location=US, id=3988fef0-fa93-4677-9230-f11ec5ab5ec2>


In [15]:
# ## Makes a Table for the chat Logs
# chat_log_table_id = f"{project_id}.{mydataset}.chat_logs"

# logging_table = f"""
# CREATE TABLE IF NOT EXISTS `{chat_log_table_id}` (
#   timestamp TIMESTAMP NOT NULL,
#   user_prompt STRING NOT NULL,
#   model_response STRING NOT NULL
# );
# """
# query_job = bq_client.query(logging_table)

# results = query_job.result()
# print(results)
# print(query_job)
# if results == None:
#   print("Big Query Client isn't working")

QueryJob<project=qwiklabs-gcp-02-47840be12dd1, location=US, id=a237f092-3020-444b-8ee4-65c2adb4a3df>


In [18]:
!bq ls {mydataset}

     tableId      Type    Labels   Time Partitioning   Clustered Fields  
 --------------- ------- -------- ------------------- ------------------ 
  chat_logs       TABLE                                                  
  faqs_table      TABLE                                                  
  withEmbedding   TABLE                                                  


## Functions

* https://www.weather.gov/documentation/services-web-api
* https://console.cloud.google.com/google/maps-apis/home;onboard=true?project=qwiklabs-gcp-03-29c3da30d8ab


### Sanatize

In [27]:
def sanatize_text(prompt):
  # Returns True (1) if is passes the sanatization test
  # Returns False (0) if it fails and needs to be disregaurded.
  user_prompt_data = modelarmor_v1.types.DataItem(text=prompt)

  ## breaks
  request = modelarmor_v1.SanitizeUserPromptRequest(
    name= armor_id,
    user_prompt_data=user_prompt_data,
  )
  response = armoredClient.sanitize_user_prompt(request=request)
  #print(response)
  #print(str(response.sanitization_result.filter_match_state))
  if response.sanitization_result.filter_match_state == 1:
    return 1
  else:
    return 0


def sanitize_response(response_text):
  # If response_text has no value, then return an error message
  if not response_text:
    return "An unknown error has occured."

  model_response_data = modelarmor_v1.DataItem(text = response_text)
  request = modelarmor_v1.SanitizeModelResponseRequest(
      name= armor_id,
      model_response_data=model_response_data,
      )

  response = genai_client.sanitize_model_response(request=request)

  # Return the Sanitized text if sensitive data was found.
  # If no sensitive data was found, just return the response text passed to the function.
  if str(response.sanitization_result.filter_match_state) == "FilterMatchState.MATCH_FOUND":
    sanitized_text = response.sanitization_result.filter_results["sdp"].sdp_filter_result.deidentify_result.data.text
    return sanitized_text
  else:
    # There was no invalid data, so just return what was sent in
    return response_text

### TOOL Function Decleration

In [28]:
rag_query_info = FunctionDeclaration(
    name="rag_query",
    description="VectorSearch's user prompt to faq database and returns related content.",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The user's question or prompt to search for in the FAQ database."
            }
        },
        "required": ["query"]
    },
)

get_forecast_info = FunctionDeclaration(
    name="get_forcast",
    description="Given a city name returns a json about upcoming weather patterns",
    parameters={
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "A city in Alaska"
            }
        },
        "required": ["city"]
    },
)


### API Functions

In [29]:
# Takes user input and return context from faq doc using rag
def rag_query(user_input):

  ## Embed user query and find closest neighbors
  instructions = (f"""
    SELECT base.answer,	base.question
    FROM VECTOR_SEARCH(
      TABLE `{embededTable}`,
      'ml_generate_embedding_result',
      (
        SELECT ml_generate_embedding_result, content AS query
        FROM ML.GENERATE_EMBEDDING(
          MODEL `{mymodel}`,
            (SELECT '{user_input}' AS content))
      ),
    top_k => 5,""" +
    """options => '{"fraction_lists_to_search": 0.01}'
    )
    """
  )
  #print(instructions)

  ## Query BQ
  res = bq_client.query(instructions)
  if res == None:
    print(":( sorry no luck")

  ## Build context
  context = ""
  for row in res:
    context += row['question']+ "\n"+ row['answer'] + ("-" * 20)

  return context

In [30]:
# Using geolocation api from gcp and the api key MY_API_KEY
# return lat long given a city in alaska
# if the city does not exsist return none.
def get_latlong(city):
  state = "alaska"
  address = f"{city}, {state}"

  geocode_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={MY_API_KEY}"

  try:
    response = requests.get(geocode_url)
    response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
    data = response.json()

    if data['status'] == 'OK' and data['results']:
      location = data['results'][0]['geometry']['location']
      lat = location['lat']
      long = location['lng']
      return lat, long
    else:
      print(f"Could not find coordinates for {city}, {state}. Status: {data['status']}")
      return None, None
  except requests.exceptions.RequestException as e:
    print(f"Error making Geocoding API call: {e}")
    return None, None

# helper function
# Given lat long coordinates return metadata wfo and x,y coordinate for forcast api call
# https://api.weather.gov/points/{lat},{long}
def get_points_metadata(lat, long):
  url = f"https://api.weather.gov/points/{lat},{long}"

  try:
    response = requests.get(url, headers={'User-Agent': 'Google Colab Weather App (your_email@example.com)'})
    response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
    data = response.json()

    # Extracting the required information
    properties = data.get('properties', {})
    wfo = properties.get('gridId')
    x = properties.get('gridX')
    y = properties.get('gridY')

    if wfo and x is not None and y is not None:
      return wfo, x, y
    else:
      print(f"Could not find WFO, gridX, or gridY for {lat},{long}. Response properties: {properties}")
      return None, None, None
  except requests.exceptions.RequestException as e:
    print(f"Error making NWS /points API call: {e}")
    return None, None, None
  y=""
  url = f"https://api.weather.gov/points/{lat},{long}"

  return wfo, x, y


# Given a city return a forcast json
# It is a list of items for different times
# "name" = tonight
# "shortForecast": "Slight Chance Light Rain",
# "detailedForecast": "A slight chance of rain. Cloudy, with a high near 34. Northeast wind ...
def get_forecast(city):

  lat_anchorage, long_anchorage = get_latlong(city)
  print (f"{lat_anchorage} {long_anchorage}")

  wfo,grid_x,grid_y = get_points_metadata(lat=lat_anchorage, long=long_anchorage)

  # Construct the forecast URL
  forecast_url = f"https://api.weather.gov/gridpoints/{wfo}/{grid_x},{grid_y}/forecast"

  # Make the API call to get the forecast
  try:
    # NWS API recommends a User-Agent header
    response = requests.get(forecast_url, headers={'User-Agent': 'Google Colab Weather App (your_email@example.com)'})
    response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
    forecast_data = response.json()
    return forecast_data['properties']['periods']
  except requests.exceptions.RequestException as e:
    print(f"Error making NWS forecast API call: {e}")
    return None



### Finalize Tool calling and Configurations


In [41]:

rag_tool = Tool(functionDeclarations=[rag_query_info])
forecast_tool = Tool(functionDeclarations=[get_forecast_info])

sys_msg = """
* You are the chat bot for Alaska's Department of Snow (ADS).
* You are friendly, but breif.
* You answer questions related to the department and weather in Alaska, if a
client asks'a question about something else remind them that you are "only the
agent for ADS, and can not speak for topics outside of that.
* Refrence Rag tool when answering FAQ about the department, there may be
information related to the topic that can be mentioned as well.
"""

config = GenerateContentConfig(
    tools=[rag_tool, forecast_tool],
    system_instruction=[sys_msg]
    )


chat = genai_client.chats.create(
   model=model,
   config=config
)


## GENERATE response function

I don't know why, but sometimes it doesn't run the first time? But I havn't been able to trigger it consistantly

In [43]:
# function calling isn't automatic. This function checks if the AI determins there needs to be
# a tool call made and makes it.
def generate(user_prompt) -> str:

  # Check if User input is safe
  if sanatize_text(user_prompt) == False:
    return "I'm sorry this prompt has flagged the security system, please ask something else."
  else:
    # log user input if safe
    cloud_logger.info(f"User Message: {user_prompt}")

  response = chat.send_message(user_prompt)

  # Do we need to make a function
  if response.candidates[0].content.parts[0].function_call:
    function_name = response.candidates[0].content.parts[0].function_call.name
    args = response.candidates[0].content.parts[0].function_call.args

    # Which function do we call?
    if function_name == "rag_query":
      context = rag_query(args["query"])
    elif function_name == "get_forcast":
      context = get_forecast(args["city"])
    else:
      context = "Unknown function call"

    # Get final prompt with proper context
    final_prompt = f"{user_prompt}\nContext: {context}"
    response = chat.send_message(final_prompt)

    # Sanatize AI response
    response = sanitize_response(response)

  # Log safe AI response and return
  cloud_logger.info(f"AI Response: {response.text}")
  return response.text

print(generate("how much training do snowplow drivers receive?"))

INFO:cloudLogger:User Message: how much training do snowplow drivers receive?
INFO:httpx:HTTP Request: POST https://us-central1-aiplatform.googleapis.com/v1beta1/projects/qwiklabs-gcp-02-47840be12dd1/locations/us-central1/publishers/google/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
INFO:cloudLogger:AI Response: ADS snowplow drivers receive specialized training in winter driving techniques, equipment maintenance, and safety protocols for extreme cold environments.


ADS snowplow drivers receive specialized training in winter driving techniques, equipment maintenance, and safety protocols for extreme cold environments.


## TESTING

In [38]:
def test_geo_api():
  cities = ["jenue", "Anchorage", "Fairbanks"]

  lat, long = get_latlong(cities[0])
  assert lat == 58.30049329999999
  assert long == -134.4201306

  lat, long = get_latlong(cities[1])
  assert lat == 61.2175758
  assert long == -149.8996785

  lat, long = get_latlong(cities[2])
  assert lat == 64.8400511
  assert long == -147.7199756

  for city in cities:
    lat, long = get_latlong(city)
    print(f"{city}: {lat}, {long}")

def test_geo_bad():
  # 63.588753, -154.4930619 <-- this is the bad location
  # because I auto the state to alaska if the city is bad it just defaults here.
  # It's just how their api is

  cities = ["Madaline", 123, "ladada!"]
  for city in cities:
    lat, long = get_latlong(city)
    assert lat == 63.588753
    assert long == -154.4930619

def test_get_points_metadata():
  city = "Anchorage"
  lat, long = get_latlong(city)
  w,x,y = get_points_metadata(lat=lat, long=long)

  print(f"{w} {x} {y}")

  lat, long = 61.2175758, -149.8996785
  w2,x2,y2 = get_points_metadata(lat=lat, long=long)
  assert w == w2
  assert x == x2
  assert y == y2

def test_get_points_metadata_bad():
  lat, long = 63.588753, -154.4930619
  w,x,y = get_points_metadata(lat=lat, long=long)
  assert w == "AFG"
  print(f"{w} {x} {y}")

def test_get_forecast():
  city_name = "Anchorage"
  forecast = get_forecast(city_name)
  print(forecast[0]['name'])
  print(forecast[0]['shortForecast'])
  print(forecast[0]['detailedForecast'])
  assert forecast is not None

## RAG test
def test_rag_query():
  user_input = "What are the requirements for a snow machine license?"
  context = rag_query(user_input)
  print(context)
  assert context is not None
  assert len(context) > 0


In [ ]:
ipytest.run('-rP')

......                                                                                       [100%]
========================================= warnings summary =========================================
../usr/local/lib/python3.12/dist-packages/_pytest/config/__init__.py:1290
  /usr/local/lib/python3.12/dist-packages/_pytest/config/__init__.py:1290: PytestAssertRewriteWarning: Module already imported so cannot be rewritten; anyio
    self._mark_plugins_for_rewrite(hook, disable_autoload)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
============================================== PASSES ==============================================
___________________________________________ test_geo_api ___________________________________________
--------------------------------------- Captured stdout call ---------------------------------------
jenue: 58.30049329999999, -134.4201306
Anchorage: 61.2175758, -149.8996785
Fairbanks: 64.8400511, -147.7199756
______________________

<ExitCode.OK: 0>

### Evaluation Test

In [ ]:
prompt_list = [
    "What is your mission?",
    "What is the weather in fairbanks tonight?",
    "what does ADS stand for?",
    "How does the DOT&PF prioritize snow removal on state roads and airports?",
    "How does the department address environmental concerns related to the use of de-icing chemicals and sand?",
    "Are there any weather alerts for Stika?",
    "how to handle avalanche emergancies",
]
eval_dataset = pd.DataFrame({
    "prompt": prompt_list,
})

#Instantiate an EvalTask by associating your dataset & selected metric
eval_task = EvalTask(
  dataset=eval_dataset,
  metrics=[MetricPromptTemplateExamples.Pointwise.GROUNDEDNESS], # optionally also check quesiton answering
)

evalresults = eval_task.evaluate(
      model=generate,
)

# Print the results
for row in evalresults.summary_metrics:
  print(f"{row} {evalresults.summary_metrics[row]}")

print(evalresults.metrics_table)


INFO:vertexai.evaluation._evaluation:Generating a total of 7 responses from the custom model function.
100%|██████████| 7/7 [00:02<00:00,  3.40it/s]
INFO:vertexai.evaluation._evaluation:All 7 responses are successfully generated from the custom model function.
INFO:vertexai.evaluation._evaluation:Multithreaded Batch Inference took: 2.067320701000426 seconds.
INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 7 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 7/7 [00:22<00:00,  3.17s/it]
INFO:vertexai.evaluation._evaluation:All 7 metric requests are successfully computed.
INFO:vertexai.evaluation._evaluation:Evaluation Took:22.20780835700134 seconds


row_count 7
groundedness/mean 0.14285714285714285
groundedness/std 0.37796447300922725
                                              prompt  \
0                              What is your mission?   
1          What is the weather in fairbanks tonight?   
2                           what does ADS stand for?   
3  How does the DOT&PF prioritize snow removal on...   
4  How does the department address environmental ...   
5            Are there any weather alerts for Stika?   
6                how to handle avalanche emergancies   

                                            response  \
0  I am a large language model, trained by Google...   
1  Tonight in Fairbanks, it will be mostly clear ...   
2  I'm sorry, I still cannot tell you what "ADS" ...   
3  As previously stated, the information availabl...   
4  I am sorry, but I still cannot find any inform...   
5  I can only check weather forecasts for valid c...   
6  I'm sorry, I still do not have information on ...   

               